<a href="https://colab.research.google.com/github/SivanandhamMani/My-Project/blob/main/Automated_Book_Publication_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install playwright langchain openai chromadb
!playwright install
!pip install langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 133.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 18.9 MB/s e

In [4]:
import os
import asyncio
import time
from playwright.async_api import async_playwright
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from chromadb import Client as ChromaClient
from transformers import pipeline
import openai
from openai import OpenAIError as e
from openai import RateLimitError
import nest_asyncio
nest_asyncio.apply()


In [6]:
from huggingface_hub import login
login(token="__")

In [8]:
!pip install huggingface_hub
from huggingface_hub import login
login("___")

In [11]:
pip install -U langchain langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.3 MB/s eta 0:00:00


In [13]:
# New (after pip installing updated package)
from langchain_openai import ChatOpenAI


In [16]:
# Constants
TARGET_URL = "https://en.wikisource.org/wiki/The_Gates_of_Morning/Book_1/Chapter_1"
SCREENSHOT_PATH = "chapter_1_screenshot.png"
RAW_TEXT_PATH = "chapter_1_raw.txt"
SPUN_TEXT_PATH = "chapter_1_spun.txt"
DB_COLLECTION = "book_content"

# Step 1: Scraping and Screenshot
async def scrape_and_screenshot():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(TARGET_URL)

        # Screenshot
        await page.screenshot(path=SCREENSHOT_PATH, full_page=True)

        # Scrape content
        content = await page.inner_text("#mw-content-text")
        with open(RAW_TEXT_PATH, "w", encoding="utf-8") as f:
            f.write(content)

        await browser.close()
        print("✅ Scraping and Screenshot Done")

from transformers import pipeline

def ai_writer_spin(raw_text, retries=3):
    delay = 10
    for attempt in range(retries):
        try:
            llm = ChatOpenAI(temperature=0.7, openai_api_key="___")
            prompt = ChatPromptTemplate.from_template("""
            You are a creative AI writer. Rewrite this chapter with improved clarity and engaging style.

            Chapter:
            {raw_text}

            Spun Version:
            """)
            messages = prompt.format_messages(raw_text=raw_text)
            spun = llm(messages).content
            if not spun:
               raise RuntimeError("Spun content is empty.")

            with open(SPUN_TEXT_PATH, "w", encoding="utf-8") as f:
                f.write(spun)
            print("✅ AI Writer Spin Done (OpenAI)")
            return spun
        except RateLimitError:
            print(f"⚠️ Rate limit hit, retrying in {delay} seconds... (attempt {attempt + 1}/{retries})")
            time.sleep(delay)
            delay *= 2  # exponential backoff
        except Exception as e:
            print(f"⚠️ Unexpected error with OpenAI: {e}")
            break  # immediately fallback if unexpected error

    print("🚨 Switching to local model (fallback)...")

    # Fallback: local model
    try:
        pipe = pipeline("text-generation", model="gpt2")  # fallback model that doesn't need auth
    except Exception as e:
        raise RuntimeError(f"❌ Local model failed: {e}")

# Step 3: Human-in-the-loop (simulate editable feedback loop)
def human_review_loop(text, iterations=2):
    for i in range(iterations):
        print(f"\n🔁 Iteration {i + 1} - Review Start")
        text = text.replace("the", "the (reviewed)", 1)  # Simulated edit
        print(f"✅ Human review simulated for iteration {i + 1}")
    return text

# Step 4: Versioning and ChromaDB Storage
def store_in_chroma(chapter_title, content):
    chroma_client = ChromaClient()
    chroma_client.create_collection(name=DB_COLLECTION)
    collection = chroma_client.get_collection(DB_COLLECTION)
    collection.add(
        documents=[content],
        ids=[chapter_title],
        metadatas=[{"version": time.time()}]
    )
    print("✅ Stored in ChromaDB")

# Step 5: Mock RL-based Search
# (In real-world, you'd train a policy using feedback. Here, we simulate.)
def rl_search(chapter_title):
    chroma_client = ChromaClient()
    collection = chroma_client.get_collection(DB_COLLECTION)
    results = collection.query(query_texts=[chapter_title], n_results=1)
    return results['documents'][0]

# Main Orchestration
async def main():
    await scrape_and_screenshot()

    with open(RAW_TEXT_PATH, encoding="utf-8") as f:
       raw_text = f.read()

    spun_text = ai_writer_spin(raw_text)
    if spun_text is None:
        print("❌ AI writer failed completely. Aborting workflow.")
        return
    reviewed_text = human_review_loop(spun_text)

    store_in_chroma("The Gates of Morning - Chapter 1", reviewed_text)

    retrieved_text = rl_search("The Gates of Morning - Chapter 1")
    print("\n🔍 Retrieved Final Version:\n")
    print(retrieved_text)

if __name__ == "__main__":
    await main()

    #asyncio.run(main())


✅ Scraping and Screenshot Done
⚠️ Rate limit hit, retrying in 10 seconds... (attempt 1/3)
⚠️ Rate limit hit, retrying in 20 seconds... (attempt 2/3)
⚠️ Rate limit hit, retrying in 40 seconds... (attempt 3/3)
🚨 Switching to local model (fallback)...


Device set to use cpu


❌ AI writer failed completely. Aborting workflow.
